# Project Euler Problem 31 ~ 40

### Hardware

In [1]:
%%bash
system_profiler SPHardwareDataType | grep -E \
"Model Identifier"\|"Processor Name"\|"Processor Speed"\
\|"Number of Processors"\|"Memory:"

      Model Identifier: MacBookPro13,1
      Processor Name: Dual-Core Intel Core i5
      Processor Speed: 2 GHz
      Number of Processors: 1
      Memory: 16 GB


In [2]:
!python -V

Python 3.7.4


### Import

In [3]:
import itertools
from functools import reduce

## Problem 31

In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation:

```
1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).
```

It is possible to make £2 in the following way:
```
1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p
```
How many different ways can £2 be made using any number of coins?


### 方針

- DP


### Python Solution

In [4]:
%%time
target = 200
coin_list = [1, 2, 5, 10, 20, 50, 100, 200]
ways = [1] + [0]*target

for coin in coin_list:
    for i in range(coin, target + 1):
        ways[i] += ways[i - coin]

ways[target]

CPU times: user 346 µs, sys: 1 µs, total: 347 µs
Wall time: 351 µs


73682

## Problem 32: Pandigital products

We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once; for example, the 5-digit number, 15234, is 1 through 5 pandigital.

The product 7254 is unusual, as the identity, 39 × 186 = 7254, containing multiplicand, multiplier, and product is 1 through 9 pandigital.

Find the sum of all products whose multiplicand/multiplier/product identity can be written as a 1 through 9 pandigital.

HINT: Some products can be obtained in more than one way so be sure to only include it once in your sum.

### Solution Guideline

<img src = "https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/project_euler/problem32.jpg?raw=true">


### Python Solution

In [5]:
%%time
def is_pandigital(num, digits):
    sep = ''
    num_string = sep.join(map(str, num))
    num_set = set(num_string)
    num_set.discard('0')
    if len(num_string) == digits and len(num_set) == digits:
        return True
    else:
        return False

def count_digits(num):
    count = 0
    while True:
        count += 1
        a, remainder = divmod(num, 10)
        if a == 0:
            return count
        else:
            num = a

base = 10
target = 9
a_max = base ** ((target - 1) // 2)
pandigitals = set()
for multiplicand in range(1, a_max):  # search over values of a
        a_len = count_digits(multiplicand)
        b_lower_digits = (target + 1) // 2 - 1  # minimum number of digits in b
        b_upper_digits = target // 2 + 1  # maximum number of digits in b
        b_lower = base ** (b_lower_digits - a_len)  # minimum value of b (half-open interval)
        b_upper = base ** (b_upper_digits - a_len)  # maximum value of b (half-open interval)

        for multiplier in range(b_lower, b_upper):  # search over values of b
            product = multiplicand * multiplier  # compute c = a * b

            if is_pandigital([multiplicand, multiplier, product], target):
                pandigitals.add(product) # save any unique target-pandigital numbers 'product'

sum(pandigitals)

CPU times: user 702 ms, sys: 7.57 ms, total: 709 ms
Wall time: 759 ms


45228

### Python Solution version 2

In [6]:
%%time
products = set()

for a in range(1,100):
    if a < 10:
        inner_range = range(1234, 10000)
    else:
        inner_range = range(123,1000)

    for b in inner_range:
        product = a * b
        if product >= 10000:
            continue
        digits = {d for d in str(product)}
        digits = digits.union({d for d in str(a)},{d for d in str(b)})
        if len(digits) == 9 and '0' not in digits:
            products.add(product)

product_sum = sum(products)
print(product_sum)

45228
CPU times: user 120 ms, sys: 4.68 ms, total: 124 ms
Wall time: 133 ms


## Problem 33

The fraction 49/98 is a curious fraction, as an inexperienced mathematician in attempting to simplify it may incorrectly believe that 49/98 = 4/8, which is correct, is obtained by cancelling the 9s.

We shall consider fractions like, 30/50 = 3/5, to be trivial examples.

There are exactly four non-trivial examples of this type of fraction, less than one in value, and containing two digits in the numerator and denominator.

If the product of these four fractions is given in its lowest common terms, find the value of the denominator.

### Solution

1. if the numerator or the denominator is mod 10, skip
2. find the common number
3. remove the common number from the denominator and the numerator, and compute the fraction (naive fraction)
4. True if the naive fraction == the true fraction else False

### Python Solution

In [7]:
%%time
def gcd(a,b):
    while b!=0:
        a,b=b,a%b
    return a

def lcm(a,b):
    return a*b//gcd(a,b)



res = []
for num in range(10, 100):
    if num % 10 == 0:
        continue
    for denom in range(num+1, 100):
        if denom % 10 == 0:
            continue
        
        num_list = list(str(num))
        denom_list = list(str(denom))
        common = set(num_list) & set(denom_list)
        if common:
            common_val = common.pop()
            num_list.remove(common_val)
            denom_list.remove(common_val)
            a = int(num_list[0])
            b = int(denom_list[0])
            if a/b == num/denom:
                res.append((a,b))

num = reduce(lambda a, b: a*b, [i[0] for i in res])
denom = reduce(lambda a, b: a*b, [i[1] for i in res])
print(denom//gcd(num, denom))

100
CPU times: user 8.46 ms, sys: 259 µs, total: 8.72 ms
Wall time: 8.98 ms


## Problem 34

145 is a curious number, as 1! + 4! + 5! = 1 + 24 + 120 = 145.

Find the sum of all numbers which are equal to the sum of the factorial of their digits.

Note: as 1! = 1 and 2! = 2 are not sums they are not included.

### Python Solution

In [8]:
%%time
a_max = reduce(lambda a, b: a*b,[i for i in range(1, 10)])*7
ans = []

for a in range(3, a_max):
    iterator = list(map(int, list(str(a))))
    res = 0
    for i in iterator:
        if i == 0:
            res += 1
        else:
            res += reduce(lambda a, b: a*b, range(1, i+1))
    if a == int(res):
        ans.append(a) 
print(sum(ans))

40730
CPU times: user 23 s, sys: 164 ms, total: 23.1 s
Wall time: 23.6 s


### Python Solution 2

In [9]:
%%time
from functools import lru_cache
from concurrent.futures import ProcessPoolExecutor

@lru_cache(maxsize=128, typed=False)
def factorial(x):
    if x < 2:
        return 1
    return x * factorial(x-1)

def is_curious(n):
    summ = 0
    for i in list(str(n)):
        summ += factorial(int(i))
    if summ == n:
        return n
    
def main():
    summ = 0
    with ProcessPoolExecutor() as executor:
        worklist = [i for i in range(3, 362880 * 9)]
        for result in executor.map(is_curious, worklist, chunksize=3000):
            if result:
                summ += result
    return summ
                

print(main())

40730
CPU times: user 2.87 s, sys: 373 ms, total: 3.24 s
Wall time: 7.38 s


## Problem 35

The number, 197, is called a circular prime because all rotations of the digits: 197, 971, and 719, are themselves prime.

There are thirteen such primes below 100: 2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79, and 97.

How many circular primes are there below one million?

### Python Solution

In [10]:
def generate_prime(max_num):
    prime_list = [2, 3]
    for i in range(5, max_num, 2):
        is_prime = True
        for x in range(2, int(i ** 0.5)+1):
            if i % x == 0:
                is_prime = False
                break
        if is_prime:
            prime_list.append(i)
    return prime_list
        
def string_rotation(num):
    res = []
    for i in range(0, len(num)):
        tmp = num[i:] + num[0:i]
        res.append(tmp)
    return set(res)
    
    

In [11]:
%%time
prime_list = list(map(str, generate_prime(1000000)))
prime_set = set(prime_list)
count = 0
emp = []
for i in prime_set:
    tmp_set = string_rotation(i)
    if tmp_set < prime_set:
        count += 1
print(count)

55
CPU times: user 4.81 s, sys: 34.9 ms, total: 4.84 s
Wall time: 4.94 s
